**Created on**: 2022.09.29

**Implemented by**: Anthony Cho

**Subject**: Linear programming: Web Mercantile

**Reference**: Introduction to operation research (10th Edition) - Hillier and Lieberman.
**Problem**: 3.4-9

## Problema: 


Web Mercantile sells many household products through an online catalog. The company needs substantial warehouse space for storing its goods. Plans now are being made for leasing warehouse storage space over the next 5 months. 

Just how much space will be required in each of these months is known. However, since these space requirements are quite different, it may be most economical to lease only the amount needed each month on a month-by-month basis. On the other hand, the additional cost for leasing space for additional months is much less than for the first month, so it may be less expensive to lease the maximum amount needed for the entire 5 months. 

Another option is the intermediate approach of changing the total amount of space leased (by adding a new lease and/or having an old lease expire) at least once but not every month.

The space requirement and the leasing costs for the various leasing periods are as follows:


| Month | Required Space (Sq.Ft)    |
|-------|---------------------------|
| 1     | 30000                     |
| 2     | 20000                     |
| 3     | 40000                     |
| 4     | 10000                     |
| 5     | 50000                     |

| Leasing period (Months)   | Cost ($) per Sq.Ft. leased     |
|---------------------------|--------------------------------|
| 1                         | 65                             |
| 2                         | 100                            |
| 3                         | 135                            |
| 4                         | 160                            |
| 5                         | 190                            |


The objective is to minimize the total leasing cost for meeting the space requirements.

**Parameters**:

* $c_j$: cost per Sq.Ft. leased for a period of $j$ month(s).   ($j=1, \ldots, 5$)
* $r_{i}$: required space (Sq.Ft) in month $i$.   ($i=1, \ldots, 5$)

**Decision variables**:
* $X_{i,j}$: amount of space  (Sq.Ft) to lease in month $i$ for a period of $j$ months. ($i=1, \ldots, 5; \; j=1, \ldots, 5-(i+1)$)

**Constraints**:
* **Requirement**: $\quad \sum_{k=1}^{i} \sum_{j=k}^{5-i+k} X_{i-k+1,j} \geq r_i, \quad$ ($i=1, \ldots, 5$).
* **All variables have to be non-negative**: $X_{i,j} \geq 0, \quad (i=1, \ldots, 5; \; j=1, \ldots, 5-(i-1))$.

**Objective function**:
* $\max_{X} \sum_{j=1}^{5} c_{j} \left( \sum_{i=1}^{5-(j-1)} \cdot X_{i,j} \right)$.


### General model

$$\max_{X} \sum_{j=1}^{5} c_{j} \left( \sum_{i=1}^{5-(j-1)} \cdot X_{i,j} \right)$$

s.a.

$$\sum_{k=1}^{i} \sum_{j=k}^{5-i+k} X_{i-k+1,j} \geq r_i, \quad (j=1, \ldots, 5).$$

$$X_{i,j} \geq 0, \quad (i=1, \ldots, 5; \; j=1, \ldots, 5-(i-1))$$

### Particular model

\begin{eqnarray*}
    \min_{X} & & 65 \left( X_{1,1} +X_{2,1} +X_{3,1} +X_{4,1} +X_{5,1} \right) \\
             & & +100 \left( X_{1,2} +X_{2,2} +X_{3,2} +X_{4,2} \right) \\
             & & +135 \left( X_{1,3} +X_{2,3} +X_{3,3} \right) \\
             & & +160 \left( X_{1,4} +X_{2,4} \right) \\
             & & +190 X_{1,5}
\end{eqnarray*}

s.a.

\begin{eqnarray*}
    X_{1,1} +X_{1,2} +X_{1,3} +X_{1,4} +X_{1,5} &\geq& 30000 \\
    X_{1,2} +X_{1,3} +X_{1,4} +X_{1,5} +X_{2,1} +X_{2,2} +X_{2,3} +X_{2,4} &\geq& 20000 \\
    X_{1,3} +X_{1,4} +X_{1,5} +X_{2,2} +X_{2,3} +X_{2,4} +X_{3,1} +X_{3,2} +X_{3,3} &\geq& 40000 \\
    X_{1,4} +X_{1,5} +X_{2,3} +X_{2,4} +X_{3,2} +X_{3,3} +X_{4,1} +X_{4,2} &\geq& 10000 \\
    X_{1,5} +X_{2,4} +X_{3,3} +X_{4,2} +X_{5,1} &\geq& 50000 \\
\end{eqnarray*}



$$X_{i,j} \geq 0, \qquad (i=1, \ldots, 5; \; j=1, \ldots, 5-(i-1))$$

In [9]:
## Library

import gurobipy as gp

### Parameters

In [10]:
## Cost per Sq.Ft. leased for a period of i months
costs = [65, 100, 135, 160, 190]

## Requirement of space (Sq.Ft) in month i.
requirements = [30000, 20000, 40000, 10000, 50000]

### LP-Model

In [11]:
## Model instance
model = gp.Model('WebMercantile')
model.modelSense = gp.GRB.MINIMIZE

In [12]:
## Decision-variables
X = {}
for i in range(1,6):
    X[i] = {}
    for j in range(1,7-i):
        X[i][j] = model.addVar(lb=0, vtype=gp.GRB.CONTINUOUS, name=f'X[{i},{j}]')

In [13]:
## Constraints

## Month 1:
lexp = sum([X[1][j] for j in range(1, 6)])
model.addConstr(lexp, gp.GRB.GREATER_EQUAL, requirements[0], name='ReqMonth[1]')

## Month 2: 
lexp = X[1][2] +X[1][3] +X[1][4] +X[1][5] +X[2][1] +X[2][2] +X[2][3] +X[2][4]
model.addConstr(lexp, gp.GRB.GREATER_EQUAL, requirements[1], name='ReqMonth[2]')

## Month 3:
lexp = X[1][3] +X[1][4] +X[1][5] +X[2][2] +X[2][3] +X[2][4] +X[3][1] +X[3][2] +X[3][3]
model.addConstr(lexp, gp.GRB.GREATER_EQUAL, requirements[2], name='ReqMonth[3]')

## Month 4:
lexp = X[1][4] +X[1][5] +X[2][3] +X[2][4] +X[3][2] +X[3][3] +X[4][1] +X[4][2]
model.addConstr(lexp, gp.GRB.GREATER_EQUAL, requirements[3], name='ReqMonth[4]')

## Month 5:
lexp = X[1][5] +X[2][4] +X[3][3] +X[4][2] +X[5][1]
model.addConstr(lexp, gp.GRB.GREATER_EQUAL, requirements[4], name='ReqMonth[5]')

## Model update
model.update()

In [14]:
## Objective function
objFun = 0
for j in range(1, 6):
    objFun += costs[j-1] * sum([ X[i][j] for i in range(1, 7-j)])

## Objective function assignment
model.setObjective(objFun)

## Model Update
model.update()

In [15]:
## Display model
model.display()

Minimize
   <gurobi.LinExpr: 65.0 X[1,1] + 100.0 X[1,2] + 135.0 X[1,3] + 160.0 X[1,4] + 190.0 X[1,5] + 65.0 X[2,1] + 100.0 X[2,2] + 135.0 X[2,3] + 160.0 X[2,4] + 65.0 X[3,1] + 100.0 X[3,2] + 135.0 X[3,3] + 65.0 X[4,1] + 100.0 X[4,2] + 65.0 X[5,1]>
Subject To
   ReqMonth[1] : <gurobi.LinExpr: X[1,1] + X[1,2] + X[1,3] + X[1,4] + X[1,5]> >= 30000.0
   ReqMonth[2] : <gurobi.LinExpr: X[1,2] + X[1,3] + X[1,4] + X[1,5] + X[2,1] + X[2,2] + X[2,3] + X[2,4]> >= 20000.0
   ReqMonth[3] : <gurobi.LinExpr: X[1,3] + X[1,4] + X[1,5] + X[2,2] + X[2,3] + X[2,4] + X[3,1] + X[3,2] + X[3,3]> >= 40000.0
   ReqMonth[4] : <gurobi.LinExpr: X[1,4] + X[1,5] + X[2,3] + X[2,4] + X[3,2] + X[3,3] + X[4,1] + X[4,2]> >= 10000.0
   ReqMonth[5] : <gurobi.LinExpr: X[1,5] + X[2,4] + X[3,3] + X[4,2] + X[5,1]> >= 50000.0


In [16]:
## Optimize model
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 5 rows, 15 columns and 35 nonzeros
Model fingerprint: 0xce606626
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 5e+04]
Presolve time: 0.01s
Presolved: 5 rows, 15 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+05   0.000000e+00      0s
       3    7.6500000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  7.650000000e+06


#### Post-Processing: Interpretation

In [17]:
if model.status == gp.GRB.OPTIMAL:
    for i in X.keys():
        for j in X[i].keys():
            if X[i][j].x:
                print('{} (Sq.Ft) will be leased in month {} for a period of {} months'.format(X[i][j].x, i, j))

    print(f'\nTotal cost ${model.ObjVal}\n')
    
    for c in model.getConstrs():
        print('Slack {:10}: {:>10.2f}'.format(c.ConstrName, c.slack))
    
else:
    print('No solution was found.')

30000.0 (Sq.Ft) will be leased in month 1 for a period of 5 months
10000.0 (Sq.Ft) will be leased in month 3 for a period of 1 months
20000.0 (Sq.Ft) will be leased in month 5 for a period of 1 months

Total cost $7650000.0

Slack ReqMonth[1]:       0.00
Slack ReqMonth[2]:  -10000.00
Slack ReqMonth[3]:       0.00
Slack ReqMonth[4]:  -20000.00
Slack ReqMonth[5]:       0.00
